In [1]:
from dotenv import load_dotenv
import os

from langchain_gigachat.chat_models import GigaChat

load_dotenv()

GIGACHAT_API_KEY = os.getenv("GIGACHAT_CREDENTIALS")
if not GIGACHAT_API_KEY:
    raise ValueError("❌ GIGACHAT_CREDENTIALS не найден в .env")

langchain_llm = GigaChat(
    credentials=GIGACHAT_API_KEY,
    verify_ssl_certs=False,
    model="GigaChat",   # при желании поменяй на Pro/Plus
    temperature=0.4,
    max_tokens=800,
)

print("✅ GigaChat LangChain LLM инициализирован")


✅ GigaChat LangChain LLM инициализирован


In [9]:
from types import SimpleNamespace
from typing import List, Dict

import autogen
from autogen import AssistantAgent, LLMConfig


class GigaChatLangChainClient:
    """
    Минимальный клиент для AG2, который внутри использует langchain_llm (GigaChat).
    """

    def __init__(self, config, **kwargs):
        self.model_name = config.get("model", "giga-chat-langchain")

    def create(self, params):
        """
        AG2 передаёт сюда messages и параметры.
        Мы вызываем langchain_llm и возвращаем объект с .choices[...].message.content.
        """
        messages: List[Dict[str, str]] = params["messages"]
        prompt = "\n".join(f"{m['role']}: {m['content']}" for m in messages)

        text = langchain_llm.predict(prompt)

        response = SimpleNamespace()
        response.model = self.model_name
        response.choices = []

        choice = SimpleNamespace()
        msg = SimpleNamespace()
        msg.content = text
        msg.function_call = None
        choice.message = msg

        response.choices.append(choice)
        return response

    def message_retrieval(self, response):
        return [choice.message.content for choice in response.choices]

    def cost(self, response) -> float:
        return 0.0

    @staticmethod
    def get_usage(response):
        return {}


print("✅ GigaChatLangChainClient определён")


✅ GigaChatLangChainClient определён


In [10]:
DATA_DESCRIPTION = """
У тебя есть таблица транзакций с такими колонками:

- transaction_id — идентификатор транзакции (целое число)
- transaction_date — дата (строка/дата)
- transaction_qty — количество товара в транзакции (число)
- store_id — идентификатор магазина
- product_id — идентификатор товара
- unit_price — цена за единицу (может быть с копейками)
- product_category — категория товара (например, Coffee, Tea, Bakery)
- product_type — тип товара (например, Brewed coffee, Chai tea, Hot chocolate)
- product_detail — конкретный SKU / название позиции (например, Ethiopia Rg, Latte Rg)

Данные физически лежат в файле 'transactions.xlsx' в текущей директории.
"""
print("✅ Описание данных подготовлено")


✅ Описание данных подготовлено


In [11]:
# Общий LLM-конфиг
llm_config = LLMConfig({
    "model": "giga-chat-langchain",
    "model_client_cls": "GigaChatLangChainClient",
})

# 1) Аналитик / Планировщик
planner_system_message = f"""
Ты аналитик по данным в продуктовой команде.

{DATA_DESCRIPTION}

Твоя задача — по текстовому запросу пользователя:

1. Формализовать требования к выборке/отчёту.
2. Ясно перечислить, какие метрики/поля должны быть в финальном pandas DataFrame.
3. При необходимости описать формулы вычисления метрик (словами/псевдокодом).
4. Описать группировки / фильтры / сортировки.

Формат ответа:
- Кратко переформулируй задачу.
- Явно перечисли шаги подготовки данных.
- Явно перечисли структуру целевого DataFrame в виде списка столбцов с пояснениями:
  Например:
  result_df:
    - store_id: идентификатор магазина
    - total_revenue: выручка = sum(transaction_qty * unit_price)
    - ...

Не пиши Python-код. Пиши только постановку задачи и формальное описание.
"""

planner = AssistantAgent(
    name="planner",
    system_message=planner_system_message,
    llm_config=llm_config,
)


# 2) Кодер
coder_system_message = f"""
Ты Python-разработчик, который пишет код для анализа данных с помощью pandas.

{DATA_DESCRIPTION}

Ты получаешь на вход формализованное ТЗ от аналитика (описание шагов и целевого DataFrame).
Твоя задача — написать ПОЛНЫЙ код на Python, который:

- Импортирует необходимые библиотеки (как минимум: import pandas as pd).
- Загружает данные из файла 'transactions.xlsx' (лист по умолчанию).
- Выполняет все фильтры, агрегации и расчёт метрик, которые описал аналитик.
- Создаёт финальный pandas DataFrame с указанной структурой и кладёт его в переменную result_df.

Требования к ответу:
- ТОЛЬКО Python-код, без пояснений, без комментариев, без markdown.
- Можно писать несколько строк кода, но без обрамления ```.
- Обязательно используй pandas.read_excel для чтения файла.
- Финальный результат должен быть в переменной result_df.

В ответ напиши целиком Python код
"""

coder = AssistantAgent(
    name="coder",
    system_message=coder_system_message,
    llm_config=llm_config,
)

# Регистрируем модельный клиент
planner.register_model_client(model_client_cls=GigaChatLangChainClient)
coder.register_model_client(model_client_cls=GigaChatLangChainClient)

print("✅ Агенты planner и coder созданы и настроены")


[autogen.oai.client: 11-29 12:09:02] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.
[autogen.oai.client: 11-29 12:09:02] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.
✅ Агенты planner и coder созданы и настроены


In [12]:
def run_two_agent_pipeline(user_query: str) -> str:
    """
    1. Пользователь формулирует запрос (user_query).
    2. Planner формализует требования и описывает целевой DataFrame.
    3. Coder на основе ответа planner'а пишет Python-код, который создаёт result_df из transactions.xlsx.
    Возвращает строку с кодом (ответ coder'а).
    """
    print("🔹 Запрос пользователя:")
    print(user_query)

    # 1) Ответ аналитика
    print("\n🧠 Ответ аналитика (planner):\n")
    planner_reply = planner.generate_reply(
        messages=[{"role": "user", "content": user_query}]
    )
    print(planner_reply)

    # 2) Промпт для кодера: передаём туда ТЗ аналитика
    coder_prompt = (
        "Ниже описано формализованное ТЗ от аналитика.\n"
        "На его основе напиши Python-код согласно своим инструкциям.\n\n"
        "=== ТЗ аналитика ===\n"
        f"{planner_reply}\n"
        "=== Конец ТЗ ===\n"
    )

    print("\n💻 Сгенерированный код (coder):\n")
    coder_reply = coder.generate_reply(
        messages=[{"role": "user", "content": coder_prompt}]
    )
    print(coder_reply)

    return coder_reply


In [13]:
user_query = """
Хочу получить таблицу по продажам кофе и чая за первую неделю января 2023 года.
Для каждой пары (store_id, product_category) нужны:
- суммарная выручка,
- суммарное количество проданных единиц товара,
- средняя цена за единицу (выручка / количество),
- количество уникальных товаров (product_id).

Сделай итоговый датафрейм с этими показателями.
"""

generated_code = run_two_agent_pipeline(user_query)


🔹 Запрос пользователя:

Хочу получить таблицу по продажам кофе и чая за первую неделю января 2023 года.
Для каждой пары (store_id, product_category) нужны:
- суммарная выручка,
- суммарное количество проданных единиц товара,
- средняя цена за единицу (выручка / количество),
- количество уникальных товаров (product_id).

Сделай итоговый датафрейм с этими показателями.


🧠 Ответ аналитика (planner):

Кратко переформулируем задачу:
Необходимо сформировать отчёт по продажам кофе и чая за первую неделю января 2023 года, сгруппированный по парам магазинов (`store_id`) и категорий товаров (`product_category`). Отчёт должен содержать следующие показатели:
- общая выручка от продаж,
- общее количество проданных единиц товара,
- среднюю цену единицы товара,
- количество уникальных товаров разных типов.

Шаги подготовки данных:
1. Загрузить данные из файла `transactions.xlsx`.
2. Фильтровать строки по дате продажи (`transaction_date`), чтобы оставить только записи первой недели января 2023 года.


In [20]:
import pandas as pd

# Чтение данных
df = pd.read_excel('transactions.xlsx')

# Фильтрация дат
# mask = (pd.to_datetime(df['transaction_date']) >= '2023-01-01') & (pd.to_datetime(df['transaction_date']) < '2023-01-08')
# df = df[mask]

# Фильтрация категорий
# categories_of_interest = ['Coffee', 'Tea']
# df = df.query("product_category.isin(@categories_of_interest)")

# Группировка и агрегирование
result_df = (df.groupby(['store_id', 'product_category'])
             .agg(total_sales=('unit_price', 'sum'),
                  total_units=('transaction_qty', 'sum'),
                  avg_price_per_unit=('unit_price', 'mean'),
                  unique_product_count=('product_id', 'nunique'))
             .reset_index())

In [21]:
result_df.head()

,store_id,product_category,total_sales,total_units,avg_price_per_unit,unique_product_count
0,3,Bakery,25844.75,7496,3.545720,11
1,3,Branded,5457.00,279,19.559140,3
2,3,Coffee,60560.30,29655,3.024235,20
3,3,Coffee beans,10219.20,502,20.356972,10
4,3,Drinking Chocolate,17822.75,6351,4.144826,4
